In [1]:
import yfinance as yf
import pandas as pd
import numpy as np

import datetime 

### Interval : Day

In [2]:
apple= yf.Ticker("aapl")
sp = yf.Ticker("^GSPC")
apple_price = apple.history(start="2018-01-01", end="2022-12-31", interval="1d")
sp_price = sp.history(start="2018-01-01", end="2022-12-31", interval="1d")

In [3]:
columns_drop = ['Dividends', 'High', 'Low', 'Volume', 'Stock Splits', 'Open']
apple_price = apple_price.drop(columns=columns_drop)
sp_price = sp_price.drop(columns=columns_drop)

In [4]:
apple_price = pd.merge(apple_price, sp_price, on="Date", how="left").reset_index()

In [5]:
pd.concat([pd.Series([1,2,3]), apple_price])

,0,Close_x,Close_y,Date
0,1.0,NaN,NaN,NaT
1,2.0,NaN,NaN,NaT
2,3.0,NaN,NaN,NaT
0,NaN,40.888069,2695.810059,2018-01-02 00:00:00-05:00
1,NaN,40.880951,2713.060059,2018-01-03 00:00:00-05:00
...,...,...,...,...
1254,NaN,131.658981,3844.820068,2022-12-23 00:00:00-05:00
1255,NaN,129.831772,3829.250000,2022-12-27 00:00:00-05:00
1256,NaN,125.847855,3783.219971,2022-12-28 00:00:00-05:00
1257,NaN,129.412415,3849.280029,2022-12-29 00:00:00-05:00


In [6]:
def addMissingDate(data): 
  missing_rows = pd.DataFrame(columns= ["Date", 'Close_x', 'Close_y'])
  first_date = data.iloc[0].Date
  last_date = data.iloc[len(data) - 1].Date
  if (first_date.weekday() != 0):
    for i in range(first_date.weekday() -1, -1, -1):
      first = data.iloc[0]
      missing_rows.loc[len(missing_rows)]= [first_date - datetime.timedelta(days=i), first.Close_x, first.Close_y ]

  if (last_date.weekday() != 4):
    for i in range(1, 4 - last_date.weekday() + 1):
      last = data.iloc[len(data) - 1]
      missing_rows.loc[len(missing_rows)]= [last_date + datetime.timedelta(days=i), last.Close_x, last.Close_y ]
  
  res = pd.concat([missing_rows, data]).sort_values('Date')

  missing_rows = pd.DataFrame(columns= ["Date", 'Close_x', 'Close_y'])
  prev = res.iloc[0].Date
  counter = 0
  for i in range(len(res)):
    cur_row = res.iloc[i]
    day_diff = (cur_row.Date - prev).days
    
    if ((cur_row.Date.weekday() == 0 & prev.weekday() == 4) or (day_diff == 1)):
      prev = cur_row.Date
      continue

    elif ((day_diff > 4) & (cur_row.Date.weekday() > 0) & (prev.weekday() < 4)):
      # print(cur_row.Date.weekday(), prev.weekday(), day_diff)
      # print("missing ending and start")
      for j in range(1, (5 - prev.weekday())):
        missing_rows.loc[len(missing_rows)]= [prev + datetime.timedelta(days=j), res.iloc[i-1].Close_x, res.iloc[i-1].Close_y ]
        counter += 1

      for k in range(0, cur_row.Date.weekday()):
        missing_rows.loc[len(missing_rows)]= [cur_row.Date - datetime.timedelta(days=k+1), cur_row.Close_x, cur_row.Close_y ]
        counter += 1
    
    elif ((cur_row.Date.weekday() > 0) & (prev.weekday() == 4)):
      # print(cur_row.Date.weekday(), prev.weekday(), day_diff)
      # print("missing start")
      for k in range(0, cur_row.Date.weekday()):
        missing_rows.loc[len(missing_rows)]= [cur_row.Date - datetime.timedelta(days=k+1), cur_row.Close_x, cur_row.Close_y ]
        counter += 1
    
    elif ((cur_row.Date.weekday() == 0) & (prev.weekday() != 4)):
      # print(cur_row.Date.weekday(), prev.weekday(), day_diff)
      # print("missing ending")
      for j in range(1, (5 - prev.weekday())):
        missing_rows.loc[len(missing_rows)]= [prev + datetime.timedelta(days=j), res.iloc[i-1].Close_x, res.iloc[i-1].Close_y ]
        counter += 1
    
    
    
       
    elif ((day_diff > 1) & (cur_row.Date.weekday() > prev.weekday()) ) :
      # print(cur_row.Date.weekday(), prev.weekday(), day_diff)
      # print("missing mid")
      for j in range(1, day_diff):
        missing_rows.loc[len(missing_rows)]= [prev + datetime.timedelta(days=j), res.iloc[i-1].Close_x, res.iloc[i-1].Close_y ]
        counter += 1

    prev = cur_row.Date
  print(counter)
  res = pd.concat([missing_rows, res]).sort_values('Date').reset_index().drop(columns="index")

  return res


In [7]:
addMissingDate(apple_price)

45


,Date,Close_x,Close_y
0,2018-01-02 00:00:00-05:00,40.888069,2695.810059
1,2018-01-02 00:00:00-05:00,40.888069,2695.810059
2,2018-01-03 00:00:00-05:00,40.880951,2713.060059
3,2018-01-04 00:00:00-05:00,41.070835,2723.989990
4,2018-01-05 00:00:00-05:00,41.538448,2743.149902
...,...,...,...
1300,2022-12-26 00:00:00-05:00,129.831772,3829.250000
1301,2022-12-27 00:00:00-05:00,129.831772,3829.250000
1302,2022-12-28 00:00:00-05:00,125.847855,3783.219971
1303,2022-12-29 00:00:00-05:00,129.412415,3849.280029


### Interval : Week


In [2]:
apple= yf.Ticker("aapl")
sp = yf.Ticker("^GSPC")
applep = apple.history(start="2019-01-01", end="2022-12-31", interval="1wk")
spp = sp.history(start="2019-01-01", end="2022-12-31", interval="1wk")

In [3]:
columns_drop = ['Dividends', 'Volume', 'Stock Splits', 'Open']
columns_drop_sp = ['Dividends', 'Volume', 'Stock Splits', 'Open', 'High', 'Low']
applep = applep.drop(columns=columns_drop)
spp = spp.drop(columns=columns_drop_sp)
applep = pd.merge(applep, spp, on="Date", how="left").reset_index()

In [4]:
applep

,Date,High,Low,Close_x,Close_y
0,2019-01-01 00:00:00-05:00,38.271118,34.211511,35.640202,2549.689941
1,2019-01-08 00:00:00-05:00,37.230303,35.782339,36.138908,2582.610107
2,2019-01-15 00:00:00-05:00,38.037413,36.150961,37.782032,2670.709961
3,2019-01-22 00:00:00-05:00,38.097649,36.548492,37.656754,2643.850098
4,2019-01-29 00:00:00-05:00,41.357383,37.129128,41.258602,2724.870117
...,...,...,...,...,...
204,2022-11-29 00:00:00-05:00,150.689917,140.136039,146.406464,3998.840088
205,2022-12-06 00:00:00-05:00,147.075443,139.786569,144.269730,3990.560059
206,2022-12-13 00:00:00-05:00,149.741373,131.119811,132.168198,3817.659912
207,2022-12-20 00:00:00-05:00,136.601432,129.442364,131.658981,3844.820068


In [5]:
import requests
import json

# set the API endpoint and parameters
symbol = 'AAPL'
api_key = 'U9NBHARV1LPZMLN0'
endpoint = f'https://www.alphavantage.co/query?function=EARNINGS&symbol={symbol}&apikey={api_key}'

# send a request to the API endpoint
response = requests.get(endpoint)

# parse the response JSON data
data = json.loads(response.content)

# extract the quarterly earnings data
earnings = data['quarterlyEarnings']

# convert the earnings data to a pandas dataframe
df = pd.DataFrame.from_dict(earnings)

# set the date column as the index and sort the dataframe by date
df = df.set_index('reportedDate').sort_index()

# filter the dataframe based on start and end dates
start_date = '2018-04-01'
end_date = '2023-03-27'
df = df.loc[start_date:end_date]

# calculate the trailing twelve months (TTM) earnings
ttm_earnings = []
for i in range(len(df)):
    if i < 3:
        ttm_earnings.append(None)
    else:
        first = float(df.iloc[i-3]['reportedEPS'])
        second = float(df.iloc[i-2]['reportedEPS'])
        third = float(df.iloc[i-1]['reportedEPS'])
        fourth = float(df.iloc[i]['reportedEPS'])
        ttm_earnings.append(first+second+third+fourth)

df['ttmEarnings'] = ttm_earnings

# calculate the PE ratio as the market price per share divided by TTM earnings
# df['peRatio'] = df['close'] / df['ttmEarnings']

# print the dataframe
print(df)

             fiscalDateEnding reportedEPS estimatedEPS surprise  \
reportedDate                                                      
2018-05-01         2018-03-31      0.6825       0.6725     0.01   
2018-07-31         2018-06-30       0.585        0.545     0.04   
2018-11-01         2018-09-30        0.73          0.7     0.03   
2019-01-29         2018-12-31        1.05         1.04     0.01   
2019-04-30         2019-03-31        0.62         0.59     0.03   
2019-07-30         2019-06-30        0.55         0.53     0.02   
2019-10-30         2019-09-30        0.76         0.71     0.05   
2020-01-28         2019-12-31        1.25         1.14     0.11   
2020-04-30         2020-03-31        0.64         0.56     0.08   
2020-07-30         2020-06-30        0.65         0.51     0.14   
2020-10-29         2020-09-30        0.73          0.7     0.03   
2021-01-27         2020-12-31        1.68         1.41     0.27   
2021-04-28         2021-03-31         1.4         0.99     0.4

In [89]:
import requests
import json

# set the API endpoint and parameters
symbol = 'AAPL'
api_key = 'U9NBHARV1LPZMLN0'

endpoint_earning = f'https://www.alphavantage.co/query?function=EARNINGS&symbol={symbol}&apikey={api_key}'
endpoint_ics = f'https://www.alphavantage.co/query?function=INCOME_STATEMENT&symbol={symbol}&apikey={api_key}'
endpoint_bs = f'https://www.alphavantage.co/query?function=BALANCE_SHEET&symbol={symbol}&apikey={api_key}'

# send a request to the API endpoint
response_e = requests.get(endpoint_earning)
response_ics = requests.get(endpoint_ics)
response_bs = requests.get(endpoint_bs)

# parse the response JSON data
data_e = json.loads(response_e.content)
data_ics = json.loads(response_ics.content)
data_bs = json.loads(response_bs.content)

# cols needed 
cols = {
    'bs': ['cashAndCashEquivalentsAtCarryingValue', 'shortLongTermDebtTotal', 'commonStockSharesOutstanding', 'totalShareholderEquity'],
    'is': ['ebitda', 'netIncome'],
    'e' : ['reportedEPS']
    }


# extract the quarterly earnings data
earnings = data_e['quarterlyEarnings']
ics = data_ics['quarterlyReports']
bs = data_bs['quarterlyReports']

# convert the earnings data to a pandas dataframe
earnings_df = pd.DataFrame.from_dict(earnings)
ics_df = pd.DataFrame.from_dict(ics)
bs_df = pd.DataFrame.from_dict(bs)

# set the date column as the index and sort the dataframe by date
earnings_df = earnings_df.set_index('reportedDate').sort_index()
ics_df = ics_df.set_index('fiscalDateEnding').sort_index()
bs_df = bs_df.set_index('fiscalDateEnding').sort_index()

# filter the dataframe based on start and end dates
start_date = '2018-04-01'
end_date = '2023-03-27'

start_date_earning = '2019-01-29'
end_date_earning = '2022-10-27'
earnings_df = earnings_df.loc[start_date_earning:end_date_earning][cols['e']]
ics_df = ics_df.loc[start_date:end_date][cols['is']]
bs_df = bs_df.loc[start_date:end_date][cols['bs']]

df = ics_df.merge(bs_df, on='fiscalDateEnding')

for col in ['ebitda','commonStockSharesOutstanding', 'shortLongTermDebtTotal', 'cashAndCashEquivalentsAtCarryingValue',
           'totalShareholderEquity', 'netIncome']:
    df[col] = df[col].astype('float')

df['debtToEquity'] = df['shortLongTermDebtTotal'] / df['totalShareholderEquity']
df['roe'] = df['netIncome'] / df['totalShareholderEquity']
df['ev/ebitda'] = (df.commonStockSharesOutstanding + df.shortLongTermDebtTotal - df.cashAndCashEquivalentsAtCarryingValue) / df.ebitda

temp_date = pd.Series(df.index).apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d'))
temp_date.index = df.index
df['date'] = temp_date
df['year'] = df['date'].apply(lambda x: x.year)
df['quarter'] = df['date'].apply(lambda x: x.month / 3)

temp_date1 = pd.Series(earnings_df.index).apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d'))
temp_date1.index = earnings_df.index
earnings_df['date'] = temp_date1
earnings_df['year'] = earnings_df['date'].apply(lambda x: x.year)
earnings_df['quarter'] = earnings_df['date'].apply(lambda x: (x.month//3) +1)


print(df)

                        ebitda     netIncome  \
fiscalDateEnding                               
2018-06-30        1.679500e+10  1.151900e+10   
2018-09-30        2.004300e+10  1.412500e+10   
2018-12-31        2.819100e+10  1.996500e+10   
2019-03-31        1.784300e+10  1.156100e+10   
2019-06-30        1.571000e+10  1.004400e+10   
2019-09-30        2.011600e+10  1.368600e+10   
2019-12-31        2.951900e+10  2.223600e+10   
2020-03-31        1.667800e+10  1.124900e+10   
2020-06-30        1.658600e+10  1.125300e+10   
2020-09-30        1.823700e+10  1.267300e+10   
2020-12-31        3.688300e+10  2.875500e+10   
2021-03-31        3.147800e+10  2.363000e+10   
2021-06-30        2.786600e+10  2.174400e+10   
2021-09-30        2.690900e+10  2.055100e+10   
2021-12-31        4.463200e+10  3.463000e+10   
2022-03-31        3.356700e+10  2.501000e+10   
2022-06-30        2.659000e+10  1.944200e+10   
2022-09-30        2.834900e+10  2.072100e+10   
2022-12-31        3.954200e+10  2.999800

In [90]:
earnings_df

,reportedEPS,date,year,quarter
reportedDate,,,,
2019-01-29,1.05,2019-01-29,2019,1
2019-04-30,0.62,2019-04-30,2019,2
2019-07-30,0.55,2019-07-30,2019,3
2019-10-30,0.76,2019-10-30,2019,4
2020-01-28,1.25,2020-01-28,2020,1
2020-04-30,0.64,2020-04-30,2020,2
2020-07-30,0.65,2020-07-30,2020,3
2020-10-29,0.73,2020-10-29,2020,4
2021-01-27,1.68,2021-01-27,2021,1


In [140]:
# def searchPrice(year, month, df):
#     date = 31
#     while True:
#         cur = df[(df['Date'].dt.day == date) & (df['Date'].dt.month == month) & (df['Date'].dt.year == year)]
#         if len(cur) == 0: 
#             date -= 1
#         else:
#             return cur.Close_x.values

In [142]:
# df['price'] = df['date'].apply(lambda x: searchPrice(x.year, x.month, applep)[0] )


In [91]:
def getData(date, var ,df):
    quarter = (int(date.month - 1) // 3) + 1
    year = date.year
    return df[(df.quarter == quarter) & (df.year == year)][var].values

In [92]:
applep_new = applep.copy()
applep_new['roe'] = applep_new['Date'].apply(lambda x: getData(x,'roe', df)[0])
applep_new['debtToEquity'] = applep_new['Date'].apply(lambda x: getData(x,'ebitda', df)[0])
applep_new['ev/ebitda']  = applep_new['Date'].apply(lambda x: getData(x,'ev/ebitda', df)[0])
applep_new['eps'] = applep_new['Date'].apply(lambda x: getData(x,'reportedEPS', earnings_df)[0])

In [93]:
applep_new

,Date,High,Low,Close_x,Close_y,roe,debtToEquity,ev/ebitda,eps
0,2019-01-01 00:00:00-05:00,38.271118,34.211511,35.640202,2549.689941,0.109210,1.784300e+10,4.457843,1.05
1,2019-01-08 00:00:00-05:00,37.230303,35.782339,36.138908,2582.610107,0.109210,1.784300e+10,4.457843,1.05
2,2019-01-15 00:00:00-05:00,38.037413,36.150961,37.782032,2670.709961,0.109210,1.784300e+10,4.457843,1.05
3,2019-01-22 00:00:00-05:00,38.097649,36.548492,37.656754,2643.850098,0.109210,1.784300e+10,4.457843,1.05
4,2019-01-29 00:00:00-05:00,41.357383,37.129128,41.258602,2724.870117,0.109210,1.784300e+10,4.457843,1.05
...,...,...,...,...,...,...,...,...,...
204,2022-11-29 00:00:00-05:00,150.689917,140.136039,146.406464,3998.840088,0.528813,3.954200e+10,2.692085,1.29
205,2022-12-06 00:00:00-05:00,147.075443,139.786569,144.269730,3990.560059,0.528813,3.954200e+10,2.692085,1.29
206,2022-12-13 00:00:00-05:00,149.741373,131.119811,132.168198,3817.659912,0.528813,3.954200e+10,2.692085,1.29
207,2022-12-20 00:00:00-05:00,136.601432,129.442364,131.658981,3844.820068,0.528813,3.954200e+10,2.692085,1.29


In [102]:
def series_to_supervised(data, target, quarterly, n_in=1, n_out=1, dropnan=True):    
    df = data.loc[:, list(pd.Series(data.columns).apply(lambda x: x not in  quarterly)) ].copy()
    n_vars = df.shape[1]
    df_quarterly = data.loc[:, list(pd.Series(data.columns).apply(lambda x: x in  quarterly))].copy()
    
    quarterly_dict= {}
    for col in quarterly:
        quarterly_dict[col] = []
        
    for i in range(len(df_quarterly)):
        for col, arr in quarterly_dict.items():
            temp = 0
            if i < n_in - 1:
                arr.append(None)
            else:
                
                for j in range(n_in):
                    temp += float(df_quarterly.iloc[i - j][col])       
                arr.append(temp / n_in)

    for col in quarterly:
        df_quarterly[col] = quarterly_dict[col]
        
    
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        if i == 1: 
          names += [ df.columns[j] + '(t)' for j in range(n_vars)]
        else:
          names += [ df.columns[j] +('(t-%d)' % (i-1)) for j in range(n_vars)]
    
    cols.append(df_quarterly.shift(n_in))
    names += list(df_quarterly.columns)
    
    
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df[target].shift(-i))
        names += [ target + ('%d(t+%d)' % (1, i+1))]
        

    # put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg


In [103]:
applepen = applep_new.copy().drop(columns=['Date'])
data = series_to_supervised(applepen, 'Close_x',['roe', 'debtToEquity', 'ev/ebitda', 'eps' ], 5, 1)
print(data)

      High(t-4)    Low(t-4)  Close_x(t-4)  Close_y(t-4)   High(t-3)  \
9     41.357383   37.129128     41.258602   2724.870117   42.299392   
10    42.299392   40.576769     40.820103   2709.800049   41.733150   
11    41.733150   40.983078     41.234715   2775.600098   42.553387   
12    42.553387   41.009689     42.156574   2796.110107   43.008282   
13    43.008282   41.793645     42.548561   2792.810059   43.339765   
..          ...         ...           ...           ...         ...   
204  156.999444  143.671940    152.852661   3871.979980  154.955949   
205  154.955949  133.952921    138.478485   3806.800049  150.050891   
206  150.050891  134.384808    148.053940   3957.250000  153.355842   
207  153.355842  145.927183    147.784348   3949.939941  151.598536   
208  151.598536  143.161422    144.000137   3963.939941  150.689917   

       Low(t-3)  Close_x(t-3)  Close_y(t-3)   High(t-2)    Low(t-2)  ...  \
9     40.576769     40.820103   2709.800049   41.733150   40.983078  ..

In [15]:
values = data.values

In [16]:
values[0]

array([  41.53843689, 2743.14990234,   42.03452682, 2786.23999023,
         42.35971832, 2810.30004883,   40.71004486, 2872.87011719,
         38.09667969, 2762.12988281,   37.12586594,   41.09529114])

In [17]:
trform_list = []

In [18]:
def minmaxscaler(row, dataset, in_size, out_size, num_var):
  trform_dict = {}
  res = np.zeros(len(row))
  for i in range(num_var): 
    cur_vars = row[[x for x in range(i, in_size * num_var, num_var)]]
    rowmaxmin = cur_vars.max() - cur_vars.min()
    rowmin = cur_vars.min()
    trform_dict[i] = {}
    trform_dict[i]['maxmin'] = rowmaxmin
    trform_dict[i]['min'] = rowmin
    res[[x for x in range(i, in_size * num_var, num_var)]] = [ (x- rowmin) / (rowmaxmin) for x in cur_vars]

    if (i == 0):
      res[-out_size:] = [ (x- rowmin) / (rowmaxmin) for x in row[-out_size:]]
  trform_list.append(trform_dict)
  return res

In [19]:
trform = np.array(list(map(lambda x: minmaxscaler(x, 'test',5,2,2), values)))

In [20]:
trform[0]

array([ 0.80734835,  0.        ,  0.92371838,  0.33217712,  1.        ,
        0.51765368,  0.61302873,  1.        ,  0.        ,  0.14631475,
       -0.22772811,  0.70339767])